# Análisis de sentimientos

En esta sección se realiza un análisis de sentimiento de tweets, como un ejemplo del uso de modelos de lenguaje previamente entrenado. En este caso, se utiliza el modelo [Distilbert](https://arxiv.org/abs/1910.01108) para el análisis de sentimientos. Este modelo es una versión más ligera del modelo. [BERT](https://arxiv.org/abs/1810.04805), que es un modelo de lenguaje previo a la entrada que se ha utilizado con muy buenos resultados en diferentes tareas de procesamiento del lenguaje natural, como [Análisis de sentimientos](https://arxiv.org/abs/1905.05583), [Clasificación de texto](https://arxiv.org/abs/1904.09077) o [Extracción de información](https://arxiv.org/abs/1906.05237).

En este caso, se utiliza el modelo previamente entrenado para el análisis de sentimientos en inglés.

## Carga del conjunto de datos`

Usaremos la librería [datasets](https://huggingface.co/docs/datasets/) Para cargar el `dataset` de los tweets. Esta La librería te permite cargar `datasets` de diferentes fuentes como [Hugging Face Hub](https://huggingface.co/datasets), [Amazon AWS](https://docs.aws.amazon.com/es_es/marketplace/latest/userguide/datasets.html) o [Google Cloud](https://cloud.google.com/ai-platform/training/docs/datasets). En este caso, cargaremos el `dataset` de tuits desde [Hugging Face Hub](https://huggingface.co/datasets/dair-ai/emotion).


In [17]:
# Instalamos las librerias que vamos a usar

%pip install transformers datasets evaluate accelerate keras scikit-learn

In [18]:
import datasets

# Cargamos el Dataset
dataset = datasets.load_dataset('dair-ai/emotion')

# Mostramos los datos de ejemplo

dataset['train'][0]

{'text': 'i didnt feel humiliated', 'label': 0}

Podemos ver que cada registro del `dataset` Contiene el texto del tweet y el sentimiento asociado. En este caso, el sentimiento está codificado con un entero entre 0 y 5, donde 0 corresponde a la tristeza, 1 a Alegria, 2 para amor, 3 a la ira, 4 para miedo y 5 para sorpresa.

## Preparación del `dataset`

En este caso, el `dataset` Ya está dividido en conjuntos de entrenamiento, prueba y validación. El siguiente paso es preparar el `dataset` para el entrenamiento modelo. En este caso, el modelo que utilizaremos es el modelo [BERT](https://arxiv.org/abs/1810.04805). Este modelo requiere que el texto sea tokenizado y los tokens estén codificados con sus identificadores numéricos correspondientes. Para hacer esto, utilizaremos un tokenizador DistilBERT previamente entrenado.

In [19]:
# Importamos el tokenizador de DistilBERT
from transformers import AutoTokenizer

# Cargamos el tokenizador
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

# Mostramos un ejemplo de tokenización
tokenizer('FC Barcelona is fucked this year')

{'input_ids': [101, 4429, 7623, 2003, 21746, 2023, 2095, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [20]:
# Definimos una función para preprocesar el texto.
# Truncamos los textos para asegurarnos de que no excedan el máximo tamaño de entrada deDistilBert

def tokenize(examples):
    return tokenizer(examples["text"], truncation=True)

PAplicar la tokenización, usaremos la función `map` de `datasets`. Esta función permite aplicar una función a cada registro del `dataset`. En este caso, la función que aplicaremos es la función `tokenize` que hemos definido anteriormente. Nosotros también usaremos `batched=True` para indicar que la función se aplicará a todo `dataset` en bloques.

In [21]:
dades_tokenitzades = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

A continuación usaremos `DataCollatorWithPadding` para crear un `DataCollator`, cuál es el objeto responsable de crear los tensores de entrada del modelo.En este caso usaremos `DataCollatorWithPadding` para crear tensores de tamaño fijo, agregando relleno a tensores más cortos.Esto es necesario para procesar los tensores en bloques.

In [22]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

## Evaluación

Para evaluar el modelo, debemos cargar el método que utilizaremos para la evaluación.En este caso usaremos la métrica `accuracy` del módulo `evaluate` de HuggingFace.

También definiremos una función para calcular las métricas del modelo. Esta función se utilizará para evaluar el modelo después de cada época.

In [23]:
import evaluate

accuracy = evaluate.load('accuracy')

In [24]:
# Definimos una función para calcular la precisión del modelo

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

## Definición de etiquetas

Antes de entrenar el modelo, debemos crear un diccionario que traduzca los identificadores numéricos del sentimiento a sus etiquetas correspondientes y viceversa.

In [25]:
id_a_etiqueta = {
    0: "SADNESS",
    1: "JOY",
    2: "LOVE",
    3: "ANGER",
    4: "FEAR",
    5: "SUPRISE"
}

etiqueta_a_id = {
    "SADNESS": 0,
    "JOY": 1,
    "LOVE": 2,
    "ANGER": 3,
    "FEAR": 4,
    "SUPRISE": 5
}

## Ajuste fino del modelo

El proceso de ajuste fino del modelo es entrenar el modelo con nuestro `dataset`. Esto permite que el modelo se adapte mejor a nuestros datos y mejore su rendimiento.

Necesitamos definir la función de optimización, el tamaño de los bloques y el número de épocas.

In [26]:
BATCH_SIZE = 16
NUM_EPOCHS = 5

In [27]:
from transformers import create_optimizer

# Definim la funció d'optimització
batches_per_epoch = len(dades_tokenitzades['train']) // BATCH_SIZE
total_steps = batches_per_epoch * NUM_EPOCHS
total_train_steps = int(total_steps * NUM_EPOCHS)
optimizer, lr_schedule = create_optimizer(
    init_lr=2e-5,
    num_train_steps=total_train_steps,
    num_warmup_steps=0,
)

Ahora podemos cargar el modelo previamente capacitado y hacer el ajuste fino. Usaremos `TFAutoModelForSequenceClassification` y agregaremos las etiquetas que hemos definido previamente.

In [28]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=len(etiqueta_a_id),
    id2label=id_a_etiqueta,
    label2id=etiqueta_a_id
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Huggingface proporciona modelos de formato `pytorch` por defecto. Para usarlos con `tensorflow` debemos usar la función `prepare_tf_dataset` del modelo.

In [29]:
tf_train_dataset = model.prepare_tf_dataset(
    dades_tokenitzades['train'],
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,
    shuffle=True
)

tf_val_dataset = model.prepare_tf_dataset(
    dades_tokenitzades['validation'],
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,
    shuffle=False
)

Para usar la función de evaluación que hemos definido previamente, debemos usar un `KerasMetricCallback` de Huggingface.Esta devolución de llamada hará la función `compute_metrics` correr después de cada época.

In [30]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_val_dataset)

Ahora podemos compilar y entrenar el modelo, utilizando el `optimizer` y la devolución de llamada que hemos definido previamente.

In [31]:
model.compile(optimizer=optimizer)
model.fit(tf_train_dataset, epochs=NUM_EPOCHS, validation_data=tf_val_dataset, callbacks=[metric_callback])

Epoch 1/5


FailedPreconditionError: Graph execution error:

Detected at node Adam/StatefulPartitionedCall_103 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/python3.11/dist-packa

## Inferencia

Para influir en el modelo, crearemos una tubería Huggingface. Esta tubería utilizará el modelo y el tokenizador que importamos anteriormente.

Luego usaremos la tubería para hacer una inferencia con un texto de ejemplo.

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
print(classifier("FC Barcelona is fucked this year"))
print(classifier("I'm not going to watch soccer again"))
print(classifier("Real Madrid is not going to treble this year. I'm relieved!"))